In [3]:
# !wget https://huggingface.co/datasets/mesolitica/malaysian-ultrachat/resolve/main/ultrachat-crossref-melayu-malay.jsonl

In [4]:
!nvidia-smi

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Thu Feb 15 10:26:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000001:00:00.0 Off |                    0 |
| N/A   43C    P0              66W / 300W |   6790MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [5]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True, device = 'cuda')

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

loading existing colbert_linear and sparse_linear---------


In [6]:
import json
import openai
import torch
from tqdm import tqdm

In [8]:
contexts, questions = [], []

with open('ultrachat-crossref-melayu-malay.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l[1]['content_ms']:
            c = l[0]['content'].strip()
            q = l[1]['content_ms'].strip()
            if not len(c) or not len(q):
                continue
            contexts.append(c)
            questions.append(q)
            
len(contexts), len(questions)

(9673, 9673)

In [9]:
contexts_v = model.encode(contexts, 
                            batch_size=12, 
                            max_length=8192,
                            )['dense_vecs']

encoding: 100%|██████████| 807/807 [05:21<00:00,  2.51it/s]


In [10]:
questions_v = model.encode(questions, 
                            batch_size=12, 
                            max_length=8192,
                            )['dense_vecs']

encoding: 100%|██████████| 807/807 [00:31<00:00, 25.81it/s]


In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [23]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}

for i in tqdm(range(len(questions_v))):
    argsort = np.argsort(cosine_similarity(questions_v[i].reshape(1, -1), contexts_v)[0])[::-1]
    for k in tops.keys():
        if i in argsort[:k]:
            tops[k] += 1

100%|██████████| 9673/9673 [12:36<00:00, 12.79it/s]


In [25]:
for k, v in tops.items():
    print(k, v / len(questions_v))

1 0.41331541403907784
3 0.669802543161377
5 0.7698749095420242
10 0.8632275405768635
